In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/news.csv


In [2]:
import pandas as pd

In [3]:
dir='../input/fake-news/news.csv'

In [4]:
dataframe=pd.read_csv(dir)

In [5]:
df=dataframe

In [6]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
df.label.value_counts()

label
REAL    3171
FAKE    3164
Name: count, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [10]:
df.describe(include='object')

,title,text,label
count,6335,6335,6335
unique,6256,6060,2
top,OnPolitics | 's politics blog,"Killing Obama administration rules, dismantlin...",REAL
freq,5,58,3171


In [11]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

<h1>No Null values present here</h1>

<h1>Remove un wanted columns</h1>

In [12]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [13]:
df

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [14]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<h1>Let's Perform some NLP in the text field using PortStemmer and also remove stopwords and url in the text </h1>

In [15]:
stem = PorterStemmer()

In [16]:
def change(text):
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.split()
    text = [stem.stem(word) for word in text if not word in stopwords.words('english')] 
    text = ' '.join(text)
    return text

In [17]:
df['text'] = df['text'].apply(change)

In [18]:
df

,title,text,label
0,You Can Smell Hillary’s Fear,daniel greenfield shillman journal fellow free...,FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,googl pinterest digg linkedin reddit stumbleup...,FAKE
2,Kerry to go to Paris in gesture of sympathy,u secretari state john f kerri said monday sto...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,kayde king kaydeek novemb lesson tonight dem l...,FAKE
4,The Battle of New York: Why This Primary Matters,primari day new york front runner hillari clin...,REAL
...,...,...,...
6330,State Department says it can't find emails fro...,state depart told republican nation committe c...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,p pb stand plutocrat pentagon post oct wikimed...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligarc...,anti trump protest tool oligarchi reform alway...,FAKE
6333,"In Ethiopia, Obama seeks progress on peace, se...",addi ababa ethiopia presid obama conven meet l...,REAL


In [19]:
x = df['text'].values
y = df['label'].values

In [20]:
vect = TfidfVectorizer()
vect.fit(x)

X = vect.transform(x)

In [21]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=2)

In [22]:
model = LogisticRegression()

In [23]:
model.fit(Xtrain,Ytrain)

LogisticRegression()

In [24]:
xtrain = model.predict(Xtrain)
print(xtrain)

['REAL' 'REAL' 'REAL' ... 'REAL' 'REAL' 'FAKE']


In [25]:
Acc = accuracy_score(xtrain,Ytrain)
print(Acc)

0.9532359905288083


In [27]:
Ytest

array(['REAL', 'FAKE', 'FAKE', ..., 'REAL', 'FAKE', 'FAKE'], dtype=object)

In [28]:
from sklearn.metrics import classification_report

In [35]:
target_name=['REAL','FAKE']

In [38]:
report=classification_report(Ytest,y_pred,target_names=target_name,output_dict=True)

In [39]:
report

{'REAL': {'precision': 0.8930099857346647,
  'recall': 0.9427710843373494,
  'f1-score': 0.9172161172161172,
  'support': 664},
 'FAKE': {'precision': 0.9328621908127208,
  'recall': 0.8756218905472637,
  'f1-score': 0.9033361847733106,
  'support': 603},
 'accuracy': 0.9108129439621152,
 'macro avg': {'precision': 0.9129360882736928,
  'recall': 0.9091964874423065,
  'f1-score': 0.9102761509947139,
  'support': 1267},
 'weighted avg': {'precision': 0.9119767415847578,
  'recall': 0.9108129439621152,
  'f1-score': 0.9106102772295249,
  'support': 1267}}

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(Ytest,y_pred)

array([[626,  38],
       [ 75, 528]])

In [30]:
y_pred=model.predict(Xtest)

In [31]:
y_pred

array(['REAL', 'FAKE', 'FAKE', ..., 'FAKE', 'FAKE', 'FAKE'], dtype=object)